## Autoencoder in Feature Engineering
Wiki 定義
- 也稱自動編碼器，是一種人工神經網絡，在無監督學習中用於有效編碼。自編碼的目的是對一組數據學習出一種表示（也稱表徵，編碼），通常用於降維。最近，自編碼的概念廣泛地用於數據的生成模型。自2010年以來，一些先進的人工智慧在深度學習網絡中採用了採用堆疊式稀疏自編碼。

簡而言之
- Autoencoder 是一個非監督式學習，透過Encoder 將原始資料(X) encode 成 latent vector，爾後 Decoder 根據 latent vector 轉換為預測值(X')，
    而預測值(X')要與原始資料(X)越相遇越好。

應用情景
- 參考這篇 [Medium: 7 Applications of Auto-Encoders every Data Scientist should know](https://towardsdatascience.com/6-applications-of-auto-encoders-every-data-scientist-should-know-dc703cbc892b)

**此次主要將著重在 Missing value 以及 特徵萃取的用途上。**

---

Autoencoder in Imputing Missing value(待完成, 搭配[論文](https://www.sciencedirect.com/science/article/pii/S2405896318320949)服用)

步驟
1. 將原始資料隨機將特徵遺失
2. 透過此資料建立Autnencoder
3. 透過真實有缺失的資料進行預測，將值補全

In [23]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

57344/57026 [==============================] - 1s 12us/step
(404, 13) (404,) (102, 13) (102,)


In [8]:
scaler = StandardScaler().fit(x_train)

In [9]:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

print(x_train.max(), x_train.min())
print(x_test.max(), x_test.min())

9.234847178400438 -3.8172503201932715
4.135832294709217 -3.512256695833765


## Autoencoder in Feature extraction

步驟
1. 透過原始資料建立一個autoencoder
2. 透過autoencoder 中的latent vector 當作合成特徵存取
3. 將合成特徵與原先特徵合併，送入模型訓練

In [10]:
# 為了對比先建立一個簡單的model去預測

rf = RandomForestRegressor().fit(x_train, y_train)
mean_squared_error(y_test, rf.predict(x_test))

13.97517111764706

In [18]:
# 建立autoencoder, 搭配: https://blog.csdn.net/hahajinbu/article/details/77982721, 抽取中間層建立一個萃取器

def get_autoencoder():
    model_input = keras.Input(shape=(13,))
    layer_one = layers.Dense(units=128, activation='relu')(model_input)
    layer_two = layers.Dense(units=64, activation='relu')(layer_one)
    latent_layer = layers.Dense(units=32, activation='relu', name='latent_layer')(layer_two)
    layer_three = layers.Dense(units=64, activation='relu')(latent_layer)
    layer_four = layers.Dense(units=128, activation='relu')(layer_three)
    model_output = layers.Dense(units=13)(layer_four)
    
    return keras.Model(model_input, model_output)

autoencoder = get_autoencoder()
autoencoder.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               1792      
_________________________________________________________________
dense_13 (Dense)             (None, 64)                8256      
_________________________________________________________________
latent_layer (Dense)         (None, 32)                2080      
_________________________________________________________________
dense_14 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_15 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_16 (Dense)             (None, 13)                1677

In [19]:
# 雖然有overfitting 但先不處理，只是簡單應用

autoencoder.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=32, epochs=100)

Epoch 1/100
13/13 [==============================] - 0s 11ms/step - loss: 576.6544 - mse: 576.6544 - mae: 22.1719 - val_loss: 580.3072 - val_mse: 580.3072 - val_mae: 22.2927
Epoch 2/100
13/13 [==============================] - 0s 3ms/step - loss: 485.7361 - mse: 485.7361 - mae: 19.9749 - val_loss: 364.0193 - val_mse: 364.0193 - val_mae: 16.8113
Epoch 3/100
13/13 [==============================] - 0s 3ms/step - loss: 198.2245 - mse: 198.2245 - mae: 11.5547 - val_loss: 114.7941 - val_mse: 114.7941 - val_mae: 8.2976
Epoch 4/100
13/13 [==============================] - 0s 4ms/step - loss: 70.3138 - mse: 70.3138 - mae: 6.2491 - val_loss: 49.1231 - val_mse: 49.1231 - val_mae: 5.4605
Epoch 5/100
13/13 [==============================] - 0s 3ms/step - loss: 39.9628 - mse: 39.9628 - mae: 4.5581 - val_loss: 30.0234 - val_mse: 30.0234 - val_mae: 4.3544
Epoch 6/100
13/13 [==============================] - 0s 4ms/step - loss: 25.9943 - mse: 25.9943 - mae: 3.6301 - val_loss: 27.1576 - val_mse: 27.157

13/13 [==============================] - 0s 3ms/step - loss: 5.0034 - mse: 5.0034 - mae: 1.6436 - val_loss: 15.6243 - val_mse: 15.6243 - val_mae: 2.5231
Epoch 51/100
13/13 [==============================] - 0s 3ms/step - loss: 4.6303 - mse: 4.6303 - mae: 1.5671 - val_loss: 14.5021 - val_mse: 14.5021 - val_mae: 2.4609
Epoch 52/100
13/13 [==============================] - 0s 4ms/step - loss: 4.3820 - mse: 4.3820 - mae: 1.5126 - val_loss: 16.6920 - val_mse: 16.6920 - val_mae: 2.6434
Epoch 53/100
13/13 [==============================] - 0s 3ms/step - loss: 4.6683 - mse: 4.6683 - mae: 1.5484 - val_loss: 15.7768 - val_mse: 15.7768 - val_mae: 2.7534
Epoch 54/100
13/13 [==============================] - 0s 4ms/step - loss: 4.7901 - mse: 4.7901 - mae: 1.6114 - val_loss: 16.2903 - val_mse: 16.2903 - val_mae: 2.7637
Epoch 55/100
13/13 [==============================] - 0s 3ms/step - loss: 4.6144 - mse: 4.6144 - mae: 1.6347 - val_loss: 15.8037 - val_mse: 15.8037 - val_mae: 2.7037
Epoch 56/100
13/1

13/13 [==============================] - 0s 3ms/step - loss: 2.0852 - mse: 2.0852 - mae: 1.0787 - val_loss: 10.5310 - val_mse: 10.5310 - val_mae: 2.2897
Epoch 100/100
13/13 [==============================] - 0s 3ms/step - loss: 1.9484 - mse: 1.9484 - mae: 1.0123 - val_loss: 11.2251 - val_mse: 11.2251 - val_mae: 2.4104


In [22]:
def get_latent_model():
    return keras.Model(autoencoder.input, autoencoder.get_layer('latent_layer').output)

latent_model = get_latent_model()
x_train_latent_vector = latent_model.predict(x_train)
x_test_latent_vector = latent_model.predict(x_test)

print(x_train_latent_vector.shape, x_test_latent_vector.shape)

(404, 32) (102, 32)


In [24]:
new_x_train = np.concatenate((x_train, x_train_latent_vector), axis=1)
new_x_test = np.concatenate((x_test, x_test_latent_vector), axis=1)

print(new_x_train.shape, new_x_test.shape)

(404, 45) (102, 45)


In [25]:
# 再用一次 RF

new_rf = RandomForestRegressor().fit(new_x_train, y_train)
mean_squared_error(y_test, new_rf.predict(new_x_test))

12.586816833333334

In [28]:
def experimental(times=100):
    count = 0
    for i in range(times):
        rf = RandomForestRegressor().fit(x_train, y_train)
        ori_mse = mean_squared_error(y_test, rf.predict(x_test))
        
        new_rf = RandomForestRegressor().fit(new_x_train, y_train)
        mse = mean_squared_error(y_test, new_rf.predict(new_x_test))
        
        if mse < ori_mse:
            count += 1
    return count

experimental(100)

93